In [1]:
! pip install mediapipe


In [2]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import time
import os

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

In [3]:
from mediapipe import solutions as mp
# 얼굴
print(dir(mp.face_detection))
# 얼굴 매쉬
print(dir(mp.face_mesh))
# 손
print(dir(mp.hands))

['FaceDetection', 'FaceKeyPoint', 'NamedTuple', 'SolutionBase', 'Union', '_FULL_RANGE_GRAPH_FILE_PATH', '_SHORT_RANGE_GRAPH_FILE_PATH', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'detection_pb2', 'enum', 'face_detection_pb2', 'get_key_point', 'location_data_pb2', 'np']
['FACEMESH_CONTOURS', 'FACEMESH_FACE_OVAL', 'FACEMESH_IRISES', 'FACEMESH_LEFT_EYE', 'FACEMESH_LEFT_EYEBROW', 'FACEMESH_LEFT_IRIS', 'FACEMESH_LIPS', 'FACEMESH_NOSE', 'FACEMESH_NUM_LANDMARKS', 'FACEMESH_NUM_LANDMARKS_WITH_IRISES', 'FACEMESH_RIGHT_EYE', 'FACEMESH_RIGHT_EYEBROW', 'FACEMESH_RIGHT_IRIS', 'FACEMESH_TESSELATION', 'FaceMesh', 'NamedTuple', 'SolutionBase', '_BINARYPB_FILE_PATH', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'association_calculator_pb2', 'constant_side_packet_calculator_pb2', 'detections_to_rects_calculator_pb2', 'gate_calculator_pb2', 'image_to_tensor_calculator_pb2', 'inferen

In [ ]:

# #Face Detection 모델 설정
# with mp_face_detection.FaceDetection(
#     model_selection=1, min_detection_confidence=0.5) as face_detection:
    
#     #웹캠 열기
#     cap = cv2.VideoCapture(0) 
#     if not cap.isOpened():
#       print("웹캠을 열 수 없습니다.")
#       exit()
#     while cap.isOpened() :
#       success, frame = cap.read()
#       if not success :
#         print("이미지를 읽을 수 없습니다.")
#         continue

#       #BGR 이미지를 RGB로 변환
#       image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

#       #얼굴 인식 처리
#       results = face_detection.process(image)

#       if results.detections:
#         # 감지된 모든 얼굴에 대해 반복
#         for detection in results.detections:
#           mp_drawing.draw_detection(frame, detection)

#         # 화면에 결과 표시
#         cv2.imshow('Face Detection', frame)

#         #'q'누르면 종료
#         key = cv2.waitKey(30) & 0xFF 
#         if key == ord('q'):
#           break
#         elif key == ord('c'):
#           timestamp = int(time.time())
#           filename = f"capture_{timestamp}.png"
#           cv2.imwrite(filename, frame)
#           print(f"이미지 저장 완료: {filename}")
        
#     cap.release()
#     cv2.destroyAllWindows()




I0000 00:00:1757311055.825306 2026061 gl_context.cc:369] GL version: 2.1 (2.1 ATI-7.0.23), renderer: AMD Radeon Pro 5500M OpenGL Engine
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1757311055.872103 2026244 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2025-09-08 14:57:36.104 python[53725:2026061] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


이미지 저장 완료: capture_1757311070.png


In [10]:
# 저장 폴더 생성
save_dir = "captures"
os.makedirs(save_dir, exist_ok=True)

# FaceDetection 초기화
with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.5) as face_detection:

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("웹캠을 열 수 없습니다.")
        exit()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("이미지를 읽을 수 없습니다.")
            continue

        # BGR -> RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # 얼굴 감지
        results = face_detection.process(image_rgb)

        # 얼굴 그리기
        if results.detections:
            for detection in results.detections:
                mp_drawing.draw_detection(frame, detection)

        # 화면 출력
        cv2.imshow('Face Detection', frame)

        # 키 입력 처리
        key = cv2.waitKey(30) & 0xFF
        if key == ord('q'):  # q 누르면 종료
            break
        elif key == ord('c'):  # c 누르면 캡처
            if results.detections:
                timestamp = int(time.time())
                h, w, _ = frame.shape

                for i, detection in enumerate(results.detections):
                    #얼굴 바운딩 박스 가져오기
                    bboxC = detection.location_data.relative_bounding_box
                    x_min = int(bboxC.xmin * w)
                    y_min = int(bboxC.ymin * h)
                    box_width = int(bboxC.width * w)
                    box_height = int(bboxC.height * h)

                    # 얼굴 영역만 잘라내기
                    face_crop = frame[
                        max(0, y_min):min(h, y_min + box_height),
                        max(0, x_min):min(w,x_min + box_width)
                    ]

                    # 파일명 지정
                    filename = os.path.join(save_dir, f"face_{timestamp}_{i}.png")
                    cv2.imwrite(filename, face_crop)
                    print(f"📸 얼굴 {i} 저장 완료: {filename}")
            else:
                print("저장할 얼굴이 없습니다.")


    cap.release()
    cv2.destroyAllWindows()



I0000 00:00:1757490210.550291   64920 gl_context.cc:369] GL version: 2.1 (2.1 ATI-7.0.23), renderer: AMD Radeon Pro 5500M OpenGL Engine
W0000 00:00:1757490210.555513  109650 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2025-09-10 16:43:32.323 python[2472:64920] error messaging the mach port for IMKCFRunLoopWakeUpReliable


In [9]:
# 저장 폴더 생성
save_dir = "captures"
os.makedirs(save_dir, exist_ok=True)

# FaceDetection 초기화
with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.5) as face_detection:

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("웹캠을 열 수 없습니다.")
        exit()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("이미지를 읽을 수 없습니다.")
            continue

        # BGR -> RGB 변환
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # 얼굴 감지
        results = face_detection.process(image_rgb)

        # 얼굴 그리기
        if results.detections:
            h, w, _ = frame.shape
            for i, detection in enumerate(results.detections):
                bboxC = detection.location_data.relative_bounding_box
                x_min = int(bboxC.xmin * w)
                y_min = int(bboxC.ymin * h)
                box_width = int(bboxC.width * w)
                box_height = int(bboxC.height * h)

                # 인식 신뢰도(%)
                score = detection.score[0]
                confidence_text = f"{int(score*100)}%"

                # 얼굴 번호 
                label = f"Person{i+1} ({confidence_text})"

                #얼굴 박스 그리기
                cv2.rectangle(frame, (x_min, y_min),(x_min + box_width, y_min + box_height), (0,255,0),2)

                # 텍스트 출력
                cv2.putText(frame, label, (x_min, y_min - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0),2 )                
                
                # 코 끝 좌표 구하기
                keypoints = detection.location_data.relative_keypoints
                nose_tip = keypoints[2]
                nose_x = int(nose_tip.x * w)
                nose_y = int(nose_tip.y * h)

                # 코 끝 찍기 (빨간 점)
                cv2. circle(frame, (nose_x, nose_y), 5, (0,0,255), -1)

                # 코 끝 좌표 텍스트
                cv2.putText(frame, f"Nose ({nose_x},{nose_y})",
                            (nose_x + 10, nose_y), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

        # 화면 출력
        cv2.imshow('Face Detection', frame)

        # 키 입력 처리
        key = cv2.waitKey(30) & 0xFF
        if key == ord('q'):  # q 누르면 종료
            break
        elif key == ord('c'):  # c 누르면 캡처
            timestamp = int(time.time())

            # 원본 프레임 저장
            frame_filename = os.path.join(save_dir, f"frame_{timestamp}.png")
            cv2.imwrite(frame_filename, frame)
            print(f"원본 프레임 저장 완료: {frame_filename}")

            # 얼굴 크롭 저장
            if results.detections:
                for i, detection in enumerate(results.detections):
                    bboxC = detection.location_data.relative_bounding_box
                    x_min = int(bboxC.xmin * w)
                    y_min = int(bboxC.ymin * h)
                    box_width = int(bboxC.width * w)
                    box_height = int(bboxC.height * h)

                    face_crop = frame[
                        max(0, y_min):min(h, y_min + box_height),
                        max(0, x_min):min(w, x_min + box_width)
                    ]

                    face_filename = os.path.join(save_dir, f"face_{timestamp}_{i}.png")
                    cv2.imwrite(face_filename, face_crop)
                    print(f"   얼굴 {i} 저장 완료: {face_filename}")
            else:
                print("얼굴이 감지되지 않아 원본 프레임만 저장되었습니다.")

    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1757489431.866311   64920 gl_context.cc:369] GL version: 2.1 (2.1 ATI-7.0.23), renderer: AMD Radeon Pro 5500M OpenGL Engine
W0000 00:00:1757489431.871546   97356 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


원본 프레임 저장 완료: captures/frame_1757489558.png
   얼굴 0 저장 완료: captures/face_1757489558_0.png
   얼굴 1 저장 완료: captures/face_1757489558_1.png
원본 프레임 저장 완료: captures/frame_1757489562.png
   얼굴 0 저장 완료: captures/face_1757489562_0.png
   얼굴 1 저장 완료: captures/face_1757489562_1.png


In [5]:
import cv2
import mediapipe as mp

mp_face_detection = mp.solutions.face_detection

cap = cv2.VideoCapture(0)
ret, frame = cap.read()
cap.release()

# BGR → RGB
image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.5) as face_detection:

    results = face_detection.process(image_rgb)

    print("\n[results 객체 속성]")
    print(dir(results))

    if results.detections:
        detection = results.detections[0]
        print("\n[첫 번째 detection 속성]")
        print(dir(detection))

        print("\n[score 값]")
        print(detection.score)

        print("\n[location_data 속성]")
        print(dir(detection.location_data))



2025-09-10 16:04:17.518 python[2472:64920] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.



[results 객체 속성]
['__add__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__match_args__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '_asdict', '_field_defaults', '_fields', '_make', '_replace', 'count', 'detections', 'index']

[첫 번째 detection 속성]
['AssociatedDetection', 'ByteSize', 'Clear', 'ClearExtension', 'ClearField', 'CopyFrom', 'DESCRIPTOR', 'DiscardUnknownFields', 'Extensions', 'FindInitializationErrors', 'FromString', 'HasExtension', 'HasField', 'IsInitialized', 'ListFields', 'MergeFrom', 'MergeFromString', 'ParseFromString', 'RegisterExtension', 'SerializePartialToString', 'SerializeToString', 'SetInParent'

I0000 00:00:1757487859.451876   64920 gl_context.cc:369] GL version: 2.1 (2.1 ATI-7.0.23), renderer: AMD Radeon Pro 5500M OpenGL Engine
W0000 00:00:1757487859.455780   77447 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
